# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import gkey

gmaps.configure(api_key=gkey)
figure_layout = {'width': '400px',
                'height': '400px',
                'border': '1px solid black',
                'padding': '1px'}

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_weather_path = "../output_data/cities.csv"

# Read the cities data:

cities_weather_df = pd.read_csv(cities_weather_path)

cities_weather_df


,City,City ID,Country,Latitute,Longitute,Date,Cloudiness,Humidity,Temp at Date,Max Temp to Date,Wind Speed
0,Cape Town,3369157,ZA,-33.93,18.42,1603154131,20,93,56.07,57.99,5.82
1,Rikitea,4030556,PF,-23.12,-134.97,1603154339,0,76,74.41,74.41,11.10
2,Busselton,2075265,AU,-33.65,115.33,1603154219,0,48,65.86,69.01,10.25
3,Upernavik,3418910,GL,72.79,-56.15,1603154213,100,93,32.81,32.81,8.03
4,Margate,2643044,GB,51.38,1.39,1603154340,100,66,52.90,54.00,5.01
...,...,...,...,...,...,...,...,...,...,...,...
561,Sibolga,1213855,ID,1.74,98.78,1603154465,47,70,80.96,80.96,1.97
562,Edd,338345,ER,13.93,41.69,1603154232,0,72,82.42,82.42,2.86
563,Conceição do Araguaia,3401845,BR,-8.26,-49.26,1603154466,100,45,82.33,82.33,1.74
564,Dhidhdhoo,1337612,MV,6.88,73.10,1603154466,17,78,81.55,81.55,13.18


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store 'Latitude' and 'Longitude' into  locations 

locations = cities_weather_df[["Latitute", "Longitute"]].astype(float)

# Convert Humidity to float and store

humidity_cities = cities_weather_df["Humidity"].astype(float)

# Create a Humidity Heatmap layer

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_cities, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

In [4]:
# Create Humility symbol layer
humidity_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Humidity: {humidity}" for humidity in humidity_cities]
)


fig = gmaps.figure()
fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = cities_weather_df.loc[(cities_weather_df["Max Temp to Date"] > 70) 
                                         & (cities_weather_df["Max Temp to Date"] < 80) 
                                         & (cities_weather_df["Cloudiness"] == 0)
                                         & (cities_weather_df["Wind Speed"] < 10),:]

narrowed_city_df = narrowed_city_df.dropna(how='any')

narrowed_city_df.reset_index(inplace=True)

del narrowed_city_df['index']

narrowed_city_df

len(narrowed_city_df)

narrowed_city_df

,City,City ID,Country,Latitute,Longitute,Date,Cloudiness,Humidity,Temp at Date,Max Temp to Date,Wind Speed
0,Hong Kong,1819729,HK,22.29,114.16,1603154265,0,74,71.20,75.00,1.01
1,Safaga,358620,EG,26.73,33.94,1603154343,0,44,77.00,77.00,4.70
2,Saraland,4088628,US,30.82,-88.07,1603154349,0,87,72.97,73.99,1.79
3,Jieshi,1805857,CN,22.81,115.83,1603154364,0,73,72.00,72.00,3.00
4,Bushehr,139817,IR,28.97,50.84,1603154381,0,53,76.05,76.05,5.44
5,Fuerte Olimpo,3438102,PY,-21.04,-57.87,1603154383,0,86,75.72,75.72,4.65
6,Mildura,2157698,AU,-34.20,142.15,1603154224,0,40,70.34,73.00,5.82
7,Benguela,3351663,AO,-12.58,13.41,1603154193,0,80,73.35,73.35,4.76
8,Bilma,2446796,NE,18.69,12.92,1603154409,0,17,77.99,77.99,3.69
9,Colonia Carlos Antonio López,3439159,PY,-26.40,-54.77,1603154412,0,67,72.84,72.84,1.25


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = []

# Create a set of lat and lon combinations

for index, row in narrowed_city_df.iterrows():
    
    lat = row["Latitute"]
    lon = row["Longitute"]

    # set up a parameters dictionary
    target_coordinates = f"{lat},{lon}"
    target_radius = 5000
    target_type = "hotel"
    
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
        }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotel_data = response.json()

    #print(json.dumps(hotel_data, indent=4, sort_keys=True))
    
    try:
        hotel_df.append(hotel_data['results'][0]['name'])
    
    except:
        hotel_df.append("") 

narrowed_city_df["First Hotel Name"] = hotel_df

narrowed_city_df = narrowed_city_df.dropna()

narrowed_city_df


,City,City ID,Country,Latitute,Longitute,Date,Cloudiness,Humidity,Temp at Date,Max Temp to Date,Wind Speed,First Hotel Name
0,Hong Kong,1819729,HK,22.29,114.16,1603154265,0,74,71.20,75.00,1.01,Hong Kong
1,Safaga,358620,EG,26.73,33.94,1603154343,0,44,77.00,77.00,4.70,Safaga
2,Saraland,4088628,US,30.82,-88.07,1603154349,0,87,72.97,73.99,1.79,Mobile
3,Jieshi,1805857,CN,22.81,115.83,1603154364,0,73,72.00,72.00,3.00,Shanwei
4,Bushehr,139817,IR,28.97,50.84,1603154381,0,53,76.05,76.05,5.44,Bandar Bushehr
5,Fuerte Olimpo,3438102,PY,-21.04,-57.87,1603154383,0,86,75.72,75.72,4.65,Fuerte Olimpo
6,Mildura,2157698,AU,-34.20,142.15,1603154224,0,40,70.34,73.00,5.82,Mildura
7,Benguela,3351663,AO,-12.58,13.41,1603154193,0,80,73.35,73.35,4.76,Benguela
8,Bilma,2446796,NE,18.69,12.92,1603154409,0,17,77.99,77.99,3.69,Bilma
9,Colonia Carlos Antonio López,3439159,PY,-26.40,-54.77,1603154412,0,67,72.84,72.84,1.25,Mayor Otaño


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{First Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Latitute", "Longitute"]]

In [8]:
# Display figure

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

fig



Figure(layout=FigureLayout(height='420px'))